In [144]:
import os

# Change directory to your target path
#os.chdir('/Users/carolinarutilidelima/Documents/OD_Sao_Paulo/')


os.chdir("/home/carolima/Documents/GitHub/OD_DC/")

In [145]:
import pyarrow.parquet as pq
import numpy as np 

In [146]:
file = "yellow_tripdata_2024-09" 

trips = pq.read_table(f'input_data/parquets/{file}.parquet')
trips = trips.to_pandas()

In [147]:
trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2024-09-01 00:05:51,2024-09-01 00:45:03,1.0,9.80,1.0,N,138,48,1,47.80,10.25,0.5,13.30,6.94,1.0,79.79,2.5,1.75
1,1,2024-09-01 00:59:35,2024-09-01 01:03:43,1.0,0.50,1.0,N,140,141,1,5.10,3.50,0.5,3.00,0.00,1.0,13.10,2.5,0.00
2,2,2024-09-01 00:25:00,2024-09-01 00:34:37,2.0,2.29,1.0,N,238,152,2,13.50,1.00,0.5,0.00,0.00,1.0,16.00,0.0,0.00
3,2,2024-09-01 00:31:00,2024-09-01 00:46:52,1.0,5.20,1.0,N,93,130,1,24.70,1.00,0.5,4.55,0.00,1.0,31.75,0.0,0.00
4,2,2024-09-01 00:11:57,2024-09-01 00:30:41,2.0,2.26,1.0,N,79,231,1,17.00,1.00,0.5,4.40,0.00,1.0,26.40,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633025,2,2024-09-30 23:14:05,2024-09-30 23:47:48,NaN,14.88,NaN,None,181,179,0,45.69,0.00,0.5,0.00,0.00,1.0,47.19,NaN,NaN
3633026,1,2024-09-30 23:29:30,2024-10-01 00:41:40,NaN,0.00,NaN,None,230,10,0,60.00,2.00,0.5,0.00,6.94,1.0,74.94,NaN,NaN
3633027,1,2024-09-30 23:48:10,2024-09-30 23:57:09,NaN,1.10,NaN,None,41,152,0,7.32,0.00,0.5,0.00,0.00,1.0,8.82,NaN,NaN
3633028,2,2024-09-30 23:02:31,2024-09-30 23:29:30,NaN,9.25,NaN,None,40,198,0,33.01,0.00,0.5,0.00,0.00,1.0,34.51,NaN,NaN


In [148]:
import pandas as pd
from geopy.distance import geodesic

# Load the uploaded Taxi stations CSV
stations_file = 'input_data/Taxi_stations.csv'
stations_data = pd.read_csv(stations_file)


In [149]:
duplicates = stations_data[stations_data.duplicated('LocationID', keep=False)]
print(duplicates)


     LocationID    Borough                                           Zone  \
55           56     Queens                                         Corona   
56           56     Queens                                         Corona   
103         103  Manhattan  Governor's Island/Ellis Island/Liberty Island   
104         103  Manhattan  Governor's Island/Ellis Island/Liberty Island   
105         103  Manhattan  Governor's Island/Ellis Island/Liberty Island   

    service_zone   latitude  longitude  
55     Boro Zone  40.741407 -73.858845  
56     Boro Zone  40.751819 -73.853582  
103  Yellow Zone  40.689860 -74.045288  
104  Yellow Zone  40.698769 -74.040771  
105  Yellow Zone  40.688784 -74.019073  


In [150]:
stations_data = stations_data.drop_duplicates(subset='LocationID', keep='first')


In [151]:
station_coords = stations_data.set_index('LocationID')[['latitude', 'longitude']].to_dict('index')


In [152]:


# Map coordinates to trips dataset
trips['pickup_coords'] = trips['PULocationID'].map(station_coords)
trips['dropoff_coords'] = trips['DOLocationID'].map(station_coords)



In [153]:
# Function to calculate distance between pickup and dropoff locations
def calculate_distance(row):
    if (
        row['pickup_coords'] and 
        row['dropoff_coords'] and
        not any(pd.isna(coord) for coord in row['pickup_coords'].values()) and
        not any(pd.isna(coord) for coord in row['dropoff_coords'].values())
    ):
        return geodesic(tuple(row['pickup_coords'].values()), tuple(row['dropoff_coords'].values())).kilometers
    return None

# Calculate distances and add to trips dataset
trips['distance_km'] = trips.apply(calculate_distance, axis=1)


In [154]:
# Filter trips with distances greater than 20 km
filtered_trips = trips[trips['distance_km'] > 30]

# Save the filtered dataset to a new CSV file
output_file_path = f'output_data/parquets_distance/{file}.csv'
filtered_trips[['PULocationID', 'DOLocationID', 'distance_km']].to_csv(output_file_path, index=False)

print(f"Filtered distances saved to: {output_file_path}")

Filtered distances saved to: output_data/parquets_distance/yellow_tripdata_2024-09.csv
